# 將普通函式（function）轉成能滿足介面（interface）的橋樑

這是 Go 中一個非常巧妙且常見的設計模式，特別在標準庫 `net/http` 中被廣泛使用。

## 核心概念

這個模式的關鍵在於：
1. 定義一個**命名函式型別** (named function type)
2. 在這個函式型別上定義**方法** (method)
3. 透過這個方法讓函式型別**實作介面** (implement interface)
4. 這樣普通函式就可以透過**型別轉換**變成滿足介面的值

## 1. Handler 介面定義

在 `net/http` 中，所有能處理 HTTP 請求的東西都要實作 `Handler` 介面：

```go
type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}
```

任何型別只要有 `ServeHTTP` 方法，就能被當成 `Handler` 使用。

In [ ]:
/* 示範：定義一個簡化版的 Handler 介面 */
package main

import "fmt"

// 簡化的 Request 和 ResponseWriter（為了示範）
type Request struct {
    Path string
}

type ResponseWriter interface {
    Write([]byte)
}

type SimpleWriter struct{}

func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("Response: %s\n", string(data))
}

// Handler 介面定義
type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

// 用 struct 實作 Handler 的傳統方式
type MyHandler struct {
    Message string
}

func (h MyHandler) ServeHTTP(w ResponseWriter, r *Request) {
    w.Write([]byte(h.Message))
}

func main() {
    handler := MyHandler{Message: "Hello from struct handler"}
    w := SimpleWriter{}
    r := &Request{Path: "/test"}
    
    handler.ServeHTTP(w, r)
    // 印出: Response: Hello from struct handler
}

## 2. 定義命名函式型別 (Named Function Type)

關鍵步驟：建立一個**命名的函式型別** `HandlerFunc`

```go
type HandlerFunc func(ResponseWriter, *Request)
```

⚠️ **重要區別**：
- `type HandlerFunc func(...)` → 命名型別，可以定義方法 ✅
- `type HandlerFunc = func(...)` → 型別別名，不能定義方法 ❌

因為 `HandlerFunc` 是命名型別，所以可以在它上面定義方法！

In [ ]:
/* 示範：定義命名函式型別 */
package main

import "fmt"

// 定義命名函式型別
type HandlerFunc func(ResponseWriter, *Request)

// 在這個型別上定義方法 - 這是關鍵！
func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    // 直接呼叫函式值本身
    f(w, r)
}

func main() {
    // 現在 HandlerFunc 有了 ServeHTTP 方法
    // 所以它實作了 Handler 介面！
    fmt.Println("HandlerFunc 現在實作了 Handler 介面")
}

## 3. 函式型別實作介面 - 橋樑的建立

當我們在 `HandlerFunc` 上定義了 `ServeHTTP` 方法後：

```go
func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)  // f 本身就是函式值，直接呼叫
}
```

這個方法做的事情很簡單：
1. 接收 `w` 和 `r` 參數
2. 呼叫 `f(w, r)` - 其中 `f` 是 `HandlerFunc` 型別的值，也就是一個函式

**結果**：`HandlerFunc` 自動實作了 `Handler` 介面！

In [ ]:
/* 示範：完整的橋樑機制 */
package main

import "fmt"

type Request struct {
    Path string
}

type ResponseWriter interface {
    Write([]byte)
}

type SimpleWriter struct{}

func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("Response: %s\n", string(data))
}

type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

type HandlerFunc func(ResponseWriter, *Request)

// 關鍵：在函式型別上定義方法
func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)  // 呼叫底層的函式
}

// 一個普通的函式
func HelloHandler(w ResponseWriter, r *Request) {
    w.Write([]byte("Hello from function!"))
}

func main() {
    // 將普通函式轉換成 HandlerFunc 型別
    var handler Handler = HandlerFunc(HelloHandler)
    
    w := SimpleWriter{}
    r := &Request{Path: "/hello"}
    
    // 呼叫介面方法
    handler.ServeHTTP(w, r)
    // 印出: Response: Hello from function!
    
    // 執行流程：
    // 1. handler.ServeHTTP(w, r) 被呼叫
    // 2. 實際呼叫 HandlerFunc.ServeHTTP()
    // 3. 內部執行 f(w, r)
    // 4. 也就是 HelloHandler(w, r)
}

## 4. 為什麼要這樣做？

### 優點 1: 簡潔性
- 不用為每個 handler 定義 struct 和 method
- 直接寫普通函式，透過型別轉換即可使用

### 優點 2: 適配器模式 (Adapter Pattern)
- 把「函式」這個東西 adapt 成符合介面的物件
- 將函式世界和介面世界連接起來

### 優點 3: 靈活性
- 可以使用閉包 (closure)
- 可以使用匿名函式
- 可以動態組合行為

In [ ]:
/* 示範：使用閉包和匿名函式的靈活性 */
package main

import "fmt"

type Request struct {
    Path string
}

type ResponseWriter interface {
    Write([]byte)
}

type SimpleWriter struct{}

func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("Response: %s\n", string(data))
}

type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

type HandlerFunc func(ResponseWriter, *Request)

func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)
}

// 工廠函式：產生帶有狀態的 handler
func MakeGreeter(name string) HandlerFunc {
    return func(w ResponseWriter, r *Request) {
        // 閉包：捕獲外部的 name 變數
        w.Write([]byte(fmt.Sprintf("Hello, %s!", name)))
    }
}

func main() {
    w := SimpleWriter{}
    r := &Request{Path: "/greet"}
    
    // 使用閉包建立不同的 handler
    greeter1 := MakeGreeter("Alice")
    greeter2 := MakeGreeter("Bob")
    
    greeter1.ServeHTTP(w, r)
    // 印出: Response: Hello, Alice!
    
    greeter2.ServeHTTP(w, r)
    // 印出: Response: Hello, Bob!
    
    // 使用匿名函式
    var handler Handler = HandlerFunc(func(w ResponseWriter, r *Request) {
        w.Write([]byte("Anonymous handler"))
    })
    
    handler.ServeHTTP(w, r)
    // 印出: Response: Anonymous handler
}

## 5. net/http 中的實際應用

### 5.1 核心函式對照

| 函式名稱 | 函式簽章 | 第二個參數型別 | 說明 |
|---------|---------|--------------|------|
| `http.Handle` | `func(pattern string, handler Handler)` | `Handler` 介面 | 接受任何實作 `Handler` 的型別 |
| `http.HandleFunc` | `func(pattern string, handler func(ResponseWriter, *Request))` | 普通函式 | 自動轉成 `HandlerFunc` 再呼叫 `Handle` |
| `http.HandlerFunc` | `type HandlerFunc func(ResponseWriter, *Request)` | N/A（這是型別定義） | 命名函式型別，實作 `Handler` 介面 |

### 5.2 為什麼需要型別轉換？

關鍵問題：**簽名相同 ≠ 型別相同**

```go
func Hello(w http.ResponseWriter, r *http.Request) {
    fmt.Fprintln(w, "Hello!")
}

// ❌ 錯誤：Hello 是普通函式，不是 Handler
http.Handle("/hello", Hello)
// 編譯錯誤: cannot use Hello (type func(...)) as type http.Handler

// ✅ 正確：透過型別轉換
http.Handle("/hello", http.HandlerFunc(Hello))
```

Go 使用 **nominal typing**（命名型別系統）：
- 兩個型別即使結構相同，只要名字不同就是不同型別
- 需要明確的型別轉換
- 介面是例外（使用 structural typing）

In [ ]:
/* 示範：模擬 http.Handle 和 http.HandleFunc 的實作 */
package main

import "fmt"

type Request struct {
    Path string
}

type ResponseWriter interface {
    Write([]byte)
}

type SimpleWriter struct{}

func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("Response: %s\n", string(data))
}

type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

type HandlerFunc func(ResponseWriter, *Request)

func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)
}

// 路由表（簡化版）
var routes = make(map[string]Handler)

// Handle: 接受 Handler 介面
func Handle(pattern string, handler Handler) {
    routes[pattern] = handler
    fmt.Printf("註冊路由: %s\n", pattern)
}

// HandleFunc: 語法糖，自動轉換
func HandleFunc(pattern string, handler func(ResponseWriter, *Request)) {
    // 內部自動轉型並呼叫 Handle
    Handle(pattern, HandlerFunc(handler))
}

// 一些 handler 函式
func Hello(w ResponseWriter, r *Request) {
    w.Write([]byte("Hello, World!"))
}

func Goodbye(w ResponseWriter, r *Request) {
    w.Write([]byte("Goodbye!"))
}

func main() {
    // 方式一：使用 Handle + 明確轉型
    Handle("/hello", HandlerFunc(Hello))
    // 印出: 註冊路由: /hello
    
    // 方式二：使用 HandleFunc（語法糖）
    HandleFunc("/goodbye", Goodbye)
    // 印出: 註冊路由: /goodbye
    
    // 測試路由
    w := SimpleWriter{}
    
    routes["/hello"].ServeHTTP(w, &Request{Path: "/hello"})
    // 印出: Response: Hello, World!
    
    routes["/goodbye"].ServeHTTP(w, &Request{Path: "/goodbye"})
    // 印出: Response: Goodbye!
}

## 6. 執行流程詳解

當你寫：
```go
http.HandleFunc("/hello", Hello)
```

實際執行流程：

```
1. http.HandleFunc("/hello", Hello)
   ↓
2. 內部展開為: http.Handle("/hello", http.HandlerFunc(Hello))
   ↓
3. 註冊到路由表: routes["/hello"] = HandlerFunc(Hello)
   ↓
4. 當請求 /hello 時
   ↓
5. 呼叫 handler.ServeHTTP(w, r)
   ↓
6. 實際執行 HandlerFunc.ServeHTTP()
   ↓
7. 內部呼叫 f(w, r)
   ↓
8. 也就是 Hello(w, r)
```

In [ ]:
/* 示範：追蹤完整的執行流程 */
package main

import "fmt"

type Request struct {
    Path string
}

type ResponseWriter interface {
    Write([]byte)
}

type SimpleWriter struct{}

func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("[6] 實際輸出: %s\n", string(data))
}

type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

type HandlerFunc func(ResponseWriter, *Request)

func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    fmt.Println("[4] HandlerFunc.ServeHTTP 被呼叫")
    fmt.Println("[5] 準備呼叫底層函式 f(w, r)")
    f(w, r)
}

var routes = make(map[string]Handler)

func Handle(pattern string, handler Handler) {
    fmt.Printf("[2] Handle: 註冊路由 %s\n", pattern)
    routes[pattern] = handler
}

func HandleFunc(pattern string, handler func(ResponseWriter, *Request)) {
    fmt.Printf("[1] HandleFunc: 收到路由 %s\n", pattern)
    Handle(pattern, HandlerFunc(handler))
}

func Hello(w ResponseWriter, r *Request) {
    fmt.Println("[5.1] Hello 函式開始執行")
    w.Write([]byte("Hello!"))
}

func main() {
    fmt.Println("=== 註冊階段 ===")
    HandleFunc("/hello", Hello)
    
    fmt.Println("\n=== 執行階段 ===")
    w := SimpleWriter{}
    r := &Request{Path: "/hello"}
    
    fmt.Println("[3] 呼叫 handler.ServeHTTP(w, r)")
    routes["/hello"].ServeHTTP(w, r)
    
    /* 完整輸出:
    === 註冊階段 ===
    [1] HandleFunc: 收到路由 /hello
    [2] Handle: 註冊路由 /hello
    
    === 執行階段 ===
    [3] 呼叫 handler.ServeHTTP(w, r)
    [4] HandlerFunc.ServeHTTP 被呼叫
    [5] 準備呼叫底層函式 f(w, r)
    [5.1] Hello 函式開始執行
    [6] 實際輸出: Hello!
    */
}

## 7. 方法集 (Method Set) 的重要細節

### 7.1 值接收者 vs 指標接收者

在定義方法時，接收者型別會影響哪些值可以呼叫這個方法：

| 方法定義 | `T` 的方法集 | `*T` 的方法集 | 可以滿足介面？ |
|---------|------------|-------------|---------------|
| `func (t T) Method()` | ✅ `Method` | ✅ `Method` | `T` 和 `*T` 都可以 |
| `func (t *T) Method()` | ❌ 無 | ✅ `Method` | 只有 `*T` 可以 |

### 7.2 net/http 使用值接收者的原因

```go
// 標準庫使用值接收者
func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)
}
```

為什麼？
- 最方便：可以直接寫 `HandlerFunc(fn)`，不用取指標
- `HandlerFunc` 本身就是函式值（引用型態），複製成本低
- 無狀態：函式值不需要修改自己

In [ ]:
/* 示範：值接收者 vs 指標接收者的差異 */
package main

import "fmt"

type Request struct{}
type ResponseWriter interface {
    Write([]byte)
}
type SimpleWriter struct{}
func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("%s\n", string(data))
}

// 介面
type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

// 版本一：值接收者（標準做法）
type HandlerFuncValue func(ResponseWriter, *Request)

func (f HandlerFuncValue) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)
}

// 版本二：指標接收者
type HandlerFuncPointer func(ResponseWriter, *Request)

func (f *HandlerFuncPointer) ServeHTTP(w ResponseWriter, r *Request) {
    (*f)(w, r)
}

func Hello(w ResponseWriter, r *Request) {
    w.Write([]byte("Hello"))
}

func main() {
    w := SimpleWriter{}
    r := &Request{}
    
    // 值接收者版本：直接使用值
    var h1 Handler = HandlerFuncValue(Hello)
    h1.ServeHTTP(w, r)
    // 印出: Hello
    
    // 指標接收者版本：需要取指標
    hfp := HandlerFuncPointer(Hello)
    var h2 Handler = &hfp  // 必須用指標
    h2.ServeHTTP(w, r)
    // 印出: Hello
    
    // ❌ 這樣會編譯錯誤（如果使用指標接收者）：
    // var h3 Handler = HandlerFuncPointer(Hello)
    // 錯誤: HandlerFuncPointer does not implement Handler
    //       (ServeHTTP method has pointer receiver)
    
    fmt.Println("值接收者更方便，不需要手動取指標")
}

## 8. 介面的內部表示

### 8.1 介面值 (Interface Value)

當你把一個值賦值給介面時，介面會儲存一個 `(type, value)` 對：

```go
var handler Handler = HandlerFunc(Hello)
```

內部結構：
```
handler = {
    type:  *HandlerFunc,      // 動態型別
    value: <Hello 函式的位址>  // 動態值
}
```

### 8.2 動態分派 (Dynamic Dispatch)

當你呼叫 `handler.ServeHTTP(w, r)` 時：
1. 查看介面的動態型別（`HandlerFunc`）
2. 在該型別的方法表中找到 `ServeHTTP`
3. 用動態值作為接收者呼叫該方法

這就是為什麼介面可以實現多型 (polymorphism)。

In [ ]:
/* 示範：觀察介面的動態型別和值 */
package main

import (
    "fmt"
    "reflect"
)

type Request struct{}
type ResponseWriter interface {
    Write([]byte)
}
type SimpleWriter struct{}
func (sw SimpleWriter) Write(data []byte) {}

type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

type HandlerFunc func(ResponseWriter, *Request)

func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)
}

func Hello(w ResponseWriter, r *Request) {
    w.Write([]byte("Hello"))
}

type MyHandler struct{}

func (h MyHandler) ServeHTTP(w ResponseWriter, r *Request) {
    w.Write([]byte("From struct"))
}

func printInterfaceInfo(name string, h Handler) {
    t := reflect.TypeOf(h)
    v := reflect.ValueOf(h)
    
    fmt.Printf("%s:\n", name)
    fmt.Printf("  動態型別: %v\n", t)
    fmt.Printf("  動態值的種類: %v\n", v.Kind())
    fmt.Println()
}

func main() {
    // 不同的實作，同一個介面
    var h1 Handler = HandlerFunc(Hello)
    var h2 Handler = MyHandler{}
    
    printInterfaceInfo("HandlerFunc", h1)
    // 印出:
    // HandlerFunc:
    //   動態型別: main.HandlerFunc
    //   動態值的種類: func
    
    printInterfaceInfo("MyHandler", h2)
    // 印出:
    // MyHandler:
    //   動態型別: main.MyHandler
    //   動態值的種類: struct
    
    fmt.Println("兩者都實作了 Handler 介面，但底層型別不同")
    fmt.Println("這就是多型 (polymorphism) 的威力！")
}

## 9. 進階應用：中介軟體 (Middleware)

這個模式最強大的應用之一是實作 HTTP middleware。

### 中介軟體的概念
- 在實際 handler 前後執行額外的邏輯
- 例如：logging、authentication、error recovery
- 可以鏈式組合多個 middleware

In [ ]:
/* 示範：實作 logging middleware */
package main

import (
    "fmt"
    "time"
)

type Request struct {
    Path string
}

type ResponseWriter interface {
    Write([]byte)
}

type SimpleWriter struct{}

func (sw SimpleWriter) Write(data []byte) {
    fmt.Printf("Response: %s\n", string(data))
}

type Handler interface {
    ServeHTTP(ResponseWriter, *Request)
}

type HandlerFunc func(ResponseWriter, *Request)

func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
    f(w, r)
}

// Middleware: 在 handler 外包一層 logging
func LoggingMiddleware(next Handler) Handler {
    return HandlerFunc(func(w ResponseWriter, r *Request) {
        start := time.Now()
        
        fmt.Printf("[LOG] 開始處理請求: %s\n", r.Path)
        
        // 呼叫下一個 handler
        next.ServeHTTP(w, r)
        
        duration := time.Since(start)
        fmt.Printf("[LOG] 完成請求: %s (耗時: %v)\n", r.Path, duration)
    })
}

// Middleware: 驗證
func AuthMiddleware(next Handler) Handler {
    return HandlerFunc(func(w ResponseWriter, r *Request) {
        fmt.Println("[AUTH] 檢查權限...")
        
        // 假設驗證通過
        fmt.Println("[AUTH] 驗證通過")
        
        next.ServeHTTP(w, r)
    })
}

// 實際的業務邏輯 handler
func BusinessHandler(w ResponseWriter, r *Request) {
    w.Write([]byte("處理業務邏輯"))
}

func main() {
    // 建立 handler 鏈：Logging -> Auth -> Business
    handler := LoggingMiddleware(
        AuthMiddleware(
            HandlerFunc(BusinessHandler),
        ),
    )
    
    w := SimpleWriter{}
    r := &Request{Path: "/api/data"}
    
    handler.ServeHTTP(w, r)
    
    /* 輸出:
    [LOG] 開始處理請求: /api/data
    [AUTH] 檢查權限...
    [AUTH] 驗證通過
    Response: 處理業務邏輯
    [LOG] 完成請求: /api/data (耗時: XXXµs)
    */
}

## 10. 總結

### 核心概念回顧

1. **命名函式型別 (Named Function Type)**
   ```go
   type HandlerFunc func(ResponseWriter, *Request)
   ```

2. **在型別上定義方法**
   ```go
   func (f HandlerFunc) ServeHTTP(w ResponseWriter, r *Request) {
       f(w, r)
   }
   ```

3. **自動實作介面**
   - `HandlerFunc` 有 `ServeHTTP` 方法
   - 因此實作了 `Handler` 介面

4. **型別轉換橋接**
   ```go
   var handler Handler = HandlerFunc(myFunction)
   ```

### 為什麼這個模式重要？

✅ **簡潔性**: 不需要為每個函式建立 struct 和 method

✅ **靈活性**: 可以使用閉包、匿名函式、函式組合

✅ **適配器模式**: 將函式世界和介面世界無縫連接

✅ **可組合性**: 容易實作 middleware、裝飾器等模式

### 關鍵要點

| 概念 | 說明 |
|-----|------|
| **Named Type** | `type F func(...)` 建立可以擁有方法的新型別 |
| **Type Alias** | `type F = func(...)` 只是別名，不能定義方法 |
| **Method Set** | 值接收者的方法屬於 `T` 和 `*T`；指標接收者只屬於 `*T` |
| **Interface Value** | 儲存 `(type, value)` 對，支援動態分派 |
| **Type Conversion** | 簽名相同不代表型別相同，需要明確轉換 |